In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from locale import atof
import locale
locale.setlocale(locale.LC_NUMERIC, '')
%matplotlib inline

In [53]:
def fetch_data(url):
    df = pd.read_html(url)[0]
#     df = pd.read_csv("static/corona.csv", index_col=False)
    if len(df.columns) >= 8:
        df.drop(df.columns.values[-1], axis=1, inplace=True)
        new_col_names = ["Country", "Cases", "NewPatients", "Deaths", "RecentDeaths",
                         "Recovered", "ActivePatients", "CriticalPatients"]
        rename_dict = dict(zip(df.columns.values, new_col_names))
        df = df.rename(columns=rename_dict)
        df["NewPatients"] = df["NewPatients"].apply(lambda x: x is not None and type(x) is not float and '+' in x and atof(x.split('+')[1]))
        df["NewPatients"] = df["NewPatients"].replace(False, 0)
        df.drop(df[df["Country"].apply(lambda x: "Total" in x)].index.values, inplace=True)
        df.fillna(value=0)
        return df
    return {"code": 404}

In [54]:
df = fetch_data("https://www.worldometers.info/coronavirus/")

In [55]:
workdf = df.sort_values("Cases", ascending=0).head(30)

In [62]:
country, activePatient, recoveryRate, deathRate = df["Country"], df["ActivePatients"], (df["Recovered"] / df["Cases"]) * 100, (df["Deaths"]/ df["Cases"]) * 100

In [75]:
data = {"Country": country,
        "ActivePatient": activePatient,
        "RecoveryRate": recoveryRate,
        "DeathRate": deathRate}

In [76]:
work = pd.DataFrame(data)

In [77]:
work.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158 entries, 0 to 157
Data columns (total 4 columns):
Country          158 non-null object
ActivePatient    158 non-null int64
RecoveryRate     74 non-null float64
DeathRate        47 non-null float64
dtypes: float64(2), int64(1), object(1)
memory usage: 11.2+ KB


In [83]:
work = work.fillna(value=0)

In [84]:
work.head()

,Country,ActivePatient,RecoveryRate,DeathRate
0,China,10719,82.785192,3.956759
1,Italy,20603,9.435487,7.309977
2,Iran,8624,32.931554,5.194432
3,S. Korea,7253,10.218084,0.918892
4,Spain,6989,6.629905,3.744550


In [95]:
fresh = work[work["ActivePatient"] > 0][ work["RecoveryRate"] > 0]

/home/alpha/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [104]:
sort_by_RR = fresh.sort_values("RecoveryRate", ascending=0)

In [107]:
sort_by_RR["RecoveryRate"]

90     90.909091
0      82.785192
17     65.517241
33     54.362416
124    50.000000
         ...    
11      0.176211
14      0.115741
13      0.112867
12      0.097656
10      0.081235
Name: RecoveryRate, Length: 72, dtype: float64